In [1]:
import pandas
import torch

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
raw_data=pandas.read_excel('data/comment_20211212.xlsx')

In [4]:
raw_sample=raw_data.head(10)

In [5]:
model=torch.load('module/bert/epoch2_0.14327293808408384_0.9603448275862069_0.9611486486486487.full')

In [6]:
torch.no_grad()

In [7]:
from transformers import BertTokenizer
pretrained_model_name='bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name,
                max_length=100,
                add_special_tokens=True,
                truncation=True,
                padding=True,
                return_tensors="pt",)

In [8]:

token=tokenizer(['颜色不好看','特别特别的不好','外观丑','不好用','不实用','服务态度差'],
                max_length=100,
                add_special_tokens=True,
                truncation=True,
                padding=True,return_tensors="pt")
token.to(device)
output=model(
    **token
    )
output[0].argmax(dim=1)

tensor([0, 1, 0, 1, 1, 2], device='cuda:0')

In [9]:
class_dict=[
    lambda x:f"apprence:{x}\n",
    lambda x:f"function:{x}\n",
    lambda x:f"service:{x}\n",
    ]
for x,y in zip(['颜色不好看','特别特别的不好','外观丑','不好用','不实用','服务态度差'],output[0].argmax(dim=1).cpu()):
    print(class_dict[y](x))

apprence:颜色不好看

function:特别特别的不好

apprence:外观丑

function:不好用

function:不实用

service:服务态度差



In [10]:
import re

partten = re.compile(
    "".join(["(.*?[", "?", "!", "？", "！", "。", "…", "\n", " ", "]|.+?$)"])
)
partten.findall("产品质感：轻盈不油腻\n产品颜色：很正。持久效果：一般 还行?")


['产品质感：轻盈不油腻\n', '产品颜色：很正。', '持久效果：一般 ', '还行?']

In [11]:
class_dict=[
    lambda x:f"apprence:{x}\n",
    lambda x:f"function:{x}\n",
    lambda x:f"service:{x}\n",
    ]
comments=raw_sample[['评论内容']].values
for paragraph in comments:
    sentences=partten.findall(paragraph[0])
    token=tokenizer(
        sentences,
        max_length=100,
        add_special_tokens=True,
        truncation=True,
        padding=True,return_tensors="pt"
    )
    token.to(device)
    output=model(
        **token
        )
    for sentence,kind in zip(sentences,output[0].argmax(dim=1).cpu()):
        print(class_dict[kind](sentence))

function:滋润效果：效果很好用原来很好看

apprence:颜色太深了，不太适合我，送人了

function:产品质感：老婆特喜欢，这个颜色适合很多性格的女人，方便携带，作为携带补妆最合适。

service:宝贝q收到就迫不及待打开，包装很安全，还送了钥匙链，礼品盒，礼品袋，卖家考虑的很周全，送人很大气，不过我是自己用的。

function:，打开口红有淡淡香味，上唇试了，图一是薄图，图二是半厚图半薄图，颜色个人比较满意，喜欢的宝宝可以果断下哦手，用的其他品牌的都是正装量，用不完过期挺可以的，迷你装的够用不用担心用不完！

function:太干了，不知道是不是假的

apprence:产品颜色：颜色正，和大只一样

apprence:看上去还不错

apprence:不错，很好看

service:打开后口红上方有人为损坏，应该是盖盖子时候伤到的，因为口红盖子内没有口红损坏的残留物，给客服说了以后给我反了十元的补贴，希望以后这些问题能改善吧

function:可以



In [12]:
import joblib
appearance_classifier = joblib.load('module/naive_bayes/multinomialNB/appearance_classifier.full')
function_classifier = joblib.load('module/naive_bayes/multinomialNB/function_classifier.full')
service_classifier = joblib.load('module/naive_bayes/multinomialNB/service_classifier.full')
appearance_vector2 = joblib.load('module/naive_bayes/multinomialNB/appearance_vector2.full')
function_vector2 = joblib.load('module/naive_bayes/multinomialNB/function_vector2.full')
service_vector2 = joblib.load('module/naive_bayes/multinomialNB/service_vector2.full')

In [13]:
from tools import *
bayes_scorer = [
    [appearance_classifier,appearance_vector2],
    [function_classifier,function_vector2],
    [service_classifier,service_vector2]
]
stopwords_list=get_stopwords_list('cn_stopwords.txt')


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.653 seconds.
Prefix dict has been built successfully.
W1231 16:48:56.394050 167199 init.cc:157] AVX is available, Please re-compile on local machine
Paddle enabled successfully......


In [21]:
def categorize_score(frame:DataFrame):
    frame['SVM向量逆——美学']=''
    frame['SVM向量逆——功能']=''
    frame['SVM向量逆——维度']=''
    for i, paragraph in enumerate(frame[["评论内容"]].values): # 又是一段脑血栓代码
        if type(paragraph[0]) != str:
            continue
        sentences = partten.findall(paragraph[0])
        token = tokenizer(
            sentences,
            max_length=100,
            add_special_tokens=True,
            truncation=True,
            padding=True,
            return_tensors="pt",
        )
        token.to(device)
        output = model(**token)
        categorized_sentences = [[], [], []]
        for sentence, kind in zip(sentences, output[0].argmax(dim=1).cpu()):
            categorized_sentences[kind].append(sentence)
        for colnmn, (class_list, scorer) in enumerate(zip(categorized_sentences, bayes_scorer)):
            if len(class_list) > 0:
                classifier, vector = scorer
                vector_ = vector.transform(
                    [seg_sentence("".join(class_list), stopwords_list)]
                )
                score = classifier.predict(vector_)
                frame.loc[i,frame.columns[5+colnmn]]=score
                frame.loc[i,frame.columns[8+colnmn]]=','.join(vector.inverse_transform(vector_)[0])

In [22]:
categorize_score(raw_data)

In [23]:
raw_data.to_excel('data/final.xlsx', engine='xlsxwriter', index=False, freeze_panes=[1,0])

In [24]:
raw_data

,productId,活动时间,评论时间,评论内容,类型,美学维度,功能维度,服务维度,SVM向量逆——美学,SVM向量逆——功能,SVM向量逆——维度
0,62383970544,2020-07-09 22:37:19,2020-04-14 20:30:43,滋润效果：效果很好用原来很好看,活动前,NaN,-1.0,NaN,,"好看,效果,效果 效果,滋润,滋润 效果",
1,62383970544,2020-07-09 22:37:19,2020-07-06 18:04:48,颜色太深了，不太适合我，送人了,活动前,1.0,NaN,NaN,"适合,颜色",,
2,62383970544,2020-07-09 22:37:19,2020-05-19 14:09:16,产品质感：老婆特喜欢，这个颜色适合很多性格的女人，方便携带，作为携带补妆最合适。,活动前,NaN,1.0,NaN,,"产品,产品 质感,喜欢,方便,最合适,老婆,质感,适合,颜色",
3,62383970544,2020-07-09 22:37:19,2020-06-16 16:54:53,宝贝q收到就迫不及待打开，包装很安全，还送了钥匙链，礼品盒，礼品袋，卖家考虑的很周全，送人很...,活动前,NaN,-1.0,1.0,,"不用,不用 担心,个人,口红,品牌,喜欢,够用,宝宝,打开,担心,比较满意,用不完,颜色,香味","包装,包装 安全,卖家,安全,宝贝,宝贝 收到,很大,打开,打开 包装,收到,考虑"
4,62383970544,2020-07-09 22:37:19,2020-05-26 15:16:10,太干了，不知道是不是假的,活动前,NaN,-1.0,NaN,,"是不是,知道,知道 是不是",
...,...,...,...,...,...,...,...,...,...,...,...
169636,100011927198,2020-04-27 17:03:52,2020-05-09 16:05:11,很好用，颜色美，大气，显白,活动后,1.0,NaN,NaN,"大气,显白,颜色,颜色 大气",,
169637,100011927198,2020-04-27 17:03:52,2020-05-01 01:43:00,准备送女友的，但是外表看就明显的做工瑕疵,活动后,-1.0,NaN,NaN,"做工,外表,明显,瑕疵",,
169638,100011927198,2020-04-27 17:03:52,2020-06-03 12:46:25,宝贝收到！物流速度真的很快。感谢京东无微不致的服务！,活动后,NaN,NaN,1.0,,,"京东,京东 服务,宝贝,宝贝 收到,感谢,收到,收到 物流,服务,物流,物流 速度,速度"
169639,100011927198,2020-04-27 17:03:52,2020-05-30 18:32:27,产品质感：很好，润润的，有巧克力的味道,活动后,NaN,-1.0,NaN,,"产品,产品 质感,味道,巧克力,巧克力 味道,润润,质感",
